# Capstone Assignment 20.1: Initial Report and Exploratory Data Analysis (EDA)

Nathan Oyama

## 1 &emsp; Planning the project

There are three data sets to accomplish this project. 

* Kaggle: "Percent Sunshine by US City". kaggle.com/datasets/thedevastator/annual-percent-of-possible-sunshine-by-us-city.

* US Geological Survey: "The United States Large-Scale Solar Photovoltaic Database (USPVDB)". US Department of the Interior. energy.usgs.gov/uspvdb/data.

* landvalue: "ZHVI 3-Bedroom Time Series($) - City". landvalue.com/research/data.

Then take the following steps _for every dataset_:

1. From the data set which is in CSV format, create a pandas DataFrame object.
1. Analyze every DataFrame and identify which columns to use for this project.
1. Format the DataFrames before merge them.

Finally, merge the three DataFrames into one.

## 2 &emsp; Analyzing Data Sets

Analyze those three data sets.

In [39]:
import pandas as pd
import re

pd.options.mode.copy_on_write = True

from sklearn.model_selection import train_test_split, GridSearchCV

import warnings
from pkg_resources import PkgResourcesDeprecationWarning

warnings.filterwarnings("ignore", category=UserWarning)

### 2.1 Analyzing Data Set 1: "Percent Sunshine by US City"

In [40]:
df_sunshine_original = pd.read_csv(
    './data/Average Percent of Possible Sunshine by US City.csv'
    )

print(df_sunshine_original.shape)
print(df_sunshine_original.head())

(317, 16)
   index           CITY JAN FEB MAR APR MAY JUN JUL AUG SEP OCT NOV DEC ANN  \
0      0  BIRMINGHAM,AL  46  53  57  65  65  67  59  62  59  66  55  49  58   
1      1  MONTGOMERY,AL  47  55  58  64  63  64  61  61  59  63  55  49  58   
2      2   ANCHORAGE,AK  43  46  51  50  51  46  43  43  41  36  35  33  43   
3      3      JUNEAU,AK  39  35  38  42  44  37  33  35  27  21  26  21  33   
4      4        NOME,AK  38  56  54  52  52  43  39  34  38  35  30  36  42   

   Unnamed: 14  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  


You can use all columns except "index" and "Unnamed: 14". The "CITY" column includes the name of the city in all uppercase, followed by a comma (",") and the state abbreviation.

In [41]:
del df_sunshine_original

### 2.2 &emsp; Analyzing Data Set 2: "The US Large-Scale Solar Photovoltaic Database (USPVDB)"

In [42]:
df_photovoltaic_original = pd.read_csv(
    './data/uspvdb_v3_0_20250430.csv'
    )

print(df_photovoltaic_original.shape)

(5712, 26)


In [43]:
print(df_photovoltaic_original.iloc[:,:13].head())

   case_id multi_poly  eia_id p_state           p_county       ylat  \
0   406374     single   66887      AK  Matanuska-Susitna  61.587349   
1   405016      multi    6304      AK   Northwest Arctic  66.838470   
2   401476      multi   60058      AL         Lauderdale  34.833809   
3   401865      multi   60679      AL               Dale  31.331732   
4   401866      multi   60680      AL            Calhoun  33.626301   

        xlong   p_area  p_img_date  p_dig_conf                   p_name  \
0 -149.789413   172005    20240814           4            Houston Solar   
1 -162.553146     8740    20240719           4          Kotzebue Hybrid   
2  -87.838394  1735134    20220212           4    River Bend Solar, LLC   
3  -85.729469   187820    20220609           4  Fort Rucker Solar Array   
4  -85.940590    39717    20210814           4         ANAD Solar Array   

   p_year p_pwr_reg  
0    2023        AK  
1    2020       NaN  
2    2016       TVA  
3    2017      SOCO  
4    2017   

In [44]:
print(df_photovoltaic_original.iloc[:,13:26].head())

  p_tech_pri p_tech_sec p_sys_type       p_axis  p_azimuth  p_tilt  p_battery  \
0         PV        NaN     ground   fixed-tilt      180.0    40.0        NaN   
1         PV        NaN     ground  single-axis      156.0    40.0  batteries   
2         PV       c-si     ground  single-axis      270.0    17.0        NaN   
3         PV  thin-film     ground  single-axis      188.0    20.0        NaN   
4         PV  thin-film     ground   fixed-tilt      180.0    20.0        NaN   

   p_cap_ac  p_cap_dc      p_type       p_agrivolt p_comm  p_zscore  
0       6.0       8.4  greenfield             crop    NaN -0.457675  
1       1.7       3.4  greenfield  non-agrivoltaic    NaN  5.617232  
2      75.0     100.2  greenfield  non-agrivoltaic    NaN -0.298527  
3      10.6      12.7  greenfield  non-agrivoltaic    NaN -0.122265  
4       7.4       9.7   superfund  non-agrivoltaic    NaN  3.031619  


In [45]:
df_photovoltaic_original.query('p_cap_ac.isnull() | p_cap_dc.isnull()').shape

(0, 26)

In [46]:
del df_photovoltaic_original

### 2.3 &emsp; Analyzing Data Set 3: "ZHVI 3-Bedroom Time Series($) - City"

In [47]:
df_landvalue_original = pd.read_csv(
    './data/City_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv.zip',
    compression='zip'
    )

df_landvalue_original.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'Metro', 'CountyName', '2000-01-31', '2000-02-29',
       ...
       '2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31',
       '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31'],
      dtype='object', length=313)

In [48]:
df_landvalue_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15711 entries, 0 to 15710
Columns: 313 entries, RegionID to 2025-05-31
dtypes: float64(305), int64(2), object(6)
memory usage: 37.5+ MB


In [49]:
print(
    df_landvalue_original[[
        'RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
        'State', 'Metro', 'CountyName', '2025-05-31'
        ]].head()
    )

   RegionID  SizeRank   RegionName RegionType StateName State  \
0      6181         0     New York       city        NY    NY   
1     12447         1  Los Angeles       city        CA    CA   
2     39051         2      Houston       city        TX    TX   
3     17426         3      Chicago       city        IL    IL   
4      6915         4  San Antonio       city        TX    TX   

                                   Metro          CountyName     2025-05-31  
0  New York-Newark-Jersey City, NY-NJ-PA       Queens County  840048.900964  
1     Los Angeles-Long Beach-Anaheim, CA  Los Angeles County  964249.977821  
2   Houston-The Woodlands-Sugar Land, TX       Harris County  253134.060059  
3     Chicago-Naperville-Elgin, IL-IN-WI         Cook County  336756.496352  
4          San Antonio-New Braunfels, TX        Bexar County  235986.092899  


In [50]:
del df_landvalue_original

### 2.4 &emsp; Planning the merged Data Frame


A combined DataFrame: `df_solar`

| Column               | Example          | Data Sets                          |
| :------------------- | :--------------- | :--------------------------------- |
| County-State         | BERKELEY,CA      | Photovoltaic, Land Value           |
| City-State           | ALAMEDA,CA       | Sunshine, Land Value               |
| Longitude            | -149.789413.     | Photovoltaic                       |
| Latitude             | 61.587349        | Sunshine, Photovoltaic, Land Value |
| Sunshine (Jan ... Dec) | 58             | Sunshine                           |
| DC                   | 6.0              | Photovoltaic                       |
| AC                   | 8.4              | Photovoltaic                       |
| Current              | 14.4             | (DC + AC)                          |
| Land Value           | 840048.900963529 | Land Value                         |

Some cities that are listed in the Land Value data set and the Sunshine data set do not have solar power plants. In later steps, you predict the current in cities that do not have solar power plants and tell which cities you should build solar power plants.


Note that in the United States, city names are unique only within the same state. Some cities share the same name and even the same county name while those are located in different states.

| Column | Land Value  | Solar   | Sunshine | (Example 1)  | (Example 2)  |
| :----- | :---------: | :-----: | :------: | :----------- | :----------- |
| City   | &#9679;     | -       | &#9679;  | Franklin     | Franklin     |
| County | &#9679;     | &#9679; | -        | Williamson   | Williamson   |
| State  | &#9679;     | &#9679; | &#9679;  | Tennessee    | Texas        |


## 3 &emsp; Extracting Data Sets

### 3.1 &emsp; Dataset 1: Sunshine

In [51]:
df_sunshine_original = pd.read_csv(
    './data/Average Percent of Possible Sunshine by US City.csv'
    )

Check the `CITY` column:

In [52]:
print(df_sunshine_original[['CITY']].value_counts(ascending=True))

CITY                  
CITY                      1
ABERDEEN,SD               2
NOME,AK                   2
NORFOLK,VA                2
NORTH PLATTE,NE           2
                         ..
GRAND RAPIDS,MI           2
GREAT FALLS,MT            2
GREEN BAY,WI              2
FLAGSTAFF,AZ              2
YAP- W CAROLINE IS.,PC    2
Name: count, Length: 159, dtype: int64


There is one invalid entry `"CITY"`, and every other city has exact two entries. Check the row where the `CITY` column is `CITY`:

In [53]:
print(df_sunshine_original[df_sunshine_original['CITY'] == 'CITY'])

     index  CITY  JAN  FEB  MAR  APR  MAY  JUN  JUL  AUG  SEP  OCT  NOV  DEC  \
158    158  CITY  JAN  FEB  MAR  APR  MAY  JUN  JUL  AUG  SEP  OCT  NOV  DEC   

     ANN  Unnamed: 14  
158  ANN          NaN  


In [54]:
print(df_sunshine_original.sort_values(by=['CITY','index']).head(10))

     index            CITY  JAN  FEB  MAR  APR  MAY  JUN JUL AUG SEP OCT NOV  \
115    115     ABERDEEN,SD  NaN   54   58   63   65   66  74  78  68  48  21   
315    315     ABERDEEN,SD  NaN   54   58   63   65   66  74  78  68  48  21   
123    123      ABILENE,TX   63   66   70   71   71   77  80  75  69  68  64   
182    182      ABILENE,TX   63   66   70   71   71   77  80  75  69  68  64   
86      86       ALBANY,NY   46   52   51   55   53   55  62  58  54  46  33   
287    287       ALBANY,NY   46   52   51   55   53   55  62  58  54  46  33   
84      84  ALBUQUERQUE,NM   73   73   73   78   80   82  76  76  77  80  75   
169    169  ALBUQUERQUE,NM   73   73   73   78   80   82  76  76  77  80  75   
108    108    ALLENTOWN,PA  NaN  NaN  NaN  NaN  NaN  NaN  90  93  82  52  47   
314    314    ALLENTOWN,PA  NaN  NaN  NaN  NaN  NaN  NaN  90  93  82  52  47   

     DEC  ANN  Unnamed: 14  
115  NaN  NaN          NaN  
315  NaN  NaN          NaN  
123   65   69          NaN  
182

In this data set, the row where `'index'` is 158 does not include appopriate values thus you can remvoe it.

See the `"index"` field of every pairs of rows for every city; one of those rows have the value of less than 158 and the other rows have the value of greater than 158. Also all other values such as `"ANN"` are the same.

Assume that this data set includes two tables with the same rows in different orders. The first table spans between line 1 and line 157 in the original CSV file, and the second table spans between line 158 and line 318. You only need the first one.

Construct a new DataFrame that is based on the original data set for the sunshine information with the following changes:

* Include all rows of the first 157 records.
* Trim off the unnecessary column `"Unnamed: 14"`.
* Split the "`CITY`" entries by the city names and the state abbreviations.

In [55]:
df_sunshine_original = pd.read_csv(
    './data/Average Percent of Possible Sunshine by US City.csv'
    )

df_sunshine = df_sunshine_original\
    .iloc[1:157]\
    .set_index(['index'])

del df_sunshine['Unnamed: 14']

df_sunshine.rename(columns={'CITY': 'City-State'}, inplace=True)

In [56]:
print(df_sunshine.head())

          City-State JAN FEB MAR APR MAY JUN JUL AUG SEP OCT NOV DEC ANN
index                                                                   
1      MONTGOMERY,AL  47  55  58  64  63  64  61  61  59  63  55  49  58
2       ANCHORAGE,AK  43  46  51  50  51  46  43  43  41  36  35  33  43
3          JUNEAU,AK  39  35  38  42  44  37  33  35  27  21  26  21  33
4            NOME,AK  38  56  54  52  52  43  39  34  38  35  30  36  42
5       FLAGSTAFF,AZ  71  73  72  82  83  88  74  75  79  77  72  76  76


In [57]:
df_sunshine

,City-State,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN
index,,,,,,,,,,,,,,
1,"MONTGOMERY,AL",47,55,58,64,63,64,61,61,59,63,55,49,58
2,"ANCHORAGE,AK",43,46,51,50,51,46,43,43,41,36,35,33,43
3,"JUNEAU,AK",39,35,38,42,44,37,33,35,27,21,26,21,33
4,"NOME,AK",38,56,54,52,52,43,39,34,38,35,30,36,42
5,"FLAGSTAFF,AZ",71,73,72,82,83,88,74,75,79,77,72,76,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,MAJURO- MARSHALL,62,66,66,55,57,55,56,62,61,55,54,54,58
153,"PAGO PAGO- AMER SAMOA,PC",43,45,50,44,38,38,45,51,57,44,42,40,44
154,"POHNPEI- CAROLINE IS.,PC",40,43,47,43,41,40,40,43,44,38,41,35,41


In [58]:
del df_sunshine_original

### 3.2 &emsp; Data Set 2: Photovoltaic

In [59]:
df_photovoltaic_original = pd.read_csv(
    './data/uspvdb_v3_0_20250430.csv'
    )

df_photovoltaic = pd.DataFrame()

df_photovoltaic = df_photovoltaic_original[[
    'case_id', 'p_county', 'p_state', 'xlong', 'ylat', 'p_cap_ac', 'p_cap_dc'
    ]].set_index('case_id')

df_photovoltaic.rename(
    columns={
        'xlong':    'Longitude',
        'ylat':     'Latitude',
        'p_cap_ac': 'AC',
        'p_cap_dc': 'DC'
        },
    inplace=True
    )

df_photovoltaic['Current'] = df_photovoltaic['AC'] + df_photovoltaic['DC'] 

df_photovoltaic['County-State'] = df_photovoltaic['p_county'].str.upper() \
    + ',' +  df_photovoltaic['p_state']
del df_photovoltaic['p_county']
del df_photovoltaic['p_state']

print(df_photovoltaic.head())

          Longitude   Latitude    AC     DC  Current          County-State
case_id                                                                   
406374  -149.789413  61.587349   6.0    8.4     14.4  MATANUSKA-SUSITNA,AK
405016  -162.553146  66.838470   1.7    3.4      5.1   NORTHWEST ARCTIC,AK
401476   -87.838394  34.833809  75.0  100.2    175.2         LAUDERDALE,AL
401865   -85.729469  31.331732  10.6   12.7     23.3               DALE,AL
401866   -85.940590  33.626301   7.4    9.7     17.1            CALHOUN,AL


### 3.3 &emsp; Dataset 3: Land Values

In this original data set, there are many columns of historical prices of the average home values of 3-bedroom houses but you only need the latest values: `"2025-05-31"`.

See the first few rows of the original data set while excluding all the other columns for historical home values:

In [60]:
df_landvalue_original = pd.read_csv(
    './data/City_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv.zip',
    compression='zip'
    )

df_landvalue = pd.DataFrame()

df_landvalue = df_landvalue_original[[
    'RegionID', 'SizeRank', 'State', 'RegionName', 'CountyName', '2025-05-31'
    ]].set_index('RegionID')

df_landvalue.rename(columns={'2025-05-31': 'Land Value'}, inplace=True)

df_landvalue['County-State']\
    = df_landvalue['CountyName']\
        .str.replace(r'\s* County$', '', regex=True)\
        .str.upper() + ',' + df_landvalue['State']
del df_landvalue['CountyName']

df_landvalue['City-State'] \
    = df_landvalue['RegionName'].str.upper() + ',' + df_landvalue['State']
del df_landvalue['RegionName']

del df_landvalue['State']

del df_landvalue_original
print(df_landvalue.head())

          SizeRank     Land Value    County-State      City-State
RegionID                                                         
6181             0  840048.900964       QUEENS,NY     NEW YORK,NY
12447            1  964249.977821  LOS ANGELES,CA  LOS ANGELES,CA
39051            2  253134.060059       HARRIS,TX      HOUSTON,TX
17426            3  336756.496352         COOK,IL      CHICAGO,IL
6915             4  235986.092899        BEXAR,TX  SAN ANTONIO,TX


## 4 &emsp; Combining Three DataFrames into One

In [61]:
df_solar = df_landvalue.copy()

df_solar = pd.merge(df_solar, df_sunshine,     on='City-State',   how='inner')
df_solar = pd.merge(df_solar, df_photovoltaic, on='County-State', how='outer')

print(df_solar.query('ANN.notnull()').head())

print(df_solar.shape)

    SizeRank     Land Value County-State City-State JAN FEB MAR APR MAY JUN  \
2       97.0  474416.339219       ADA,ID   BOISE,ID  32  49  66  68  74  76   
3       97.0  474416.339219       ADA,ID   BOISE,ID  32  49  66  68  74  76   
63     246.0  314400.663535    ALBANY,NY  ALBANY,NY  46  52  51  55  53  55   
64     246.0  314400.663535    ALBANY,NY  ALBANY,NY  46  52  51  55  53  55   
65     246.0  314400.663535    ALBANY,NY  ALBANY,NY  46  52  51  55  53  55   

    ... SEP OCT NOV DEC ANN   Longitude   Latitude    AC    DC  Current  
2   ...  80  69  41  34  63 -116.327415  43.438301  40.0  54.6     94.6  
3   ...  80  69  41  34  63 -116.289497  43.468910  20.0  26.0     46.0  
63  ...  54  46  33  36  50  -73.865364  42.585896   2.0   2.3      4.3  
64  ...  54  46  33  36  50  -73.826706  42.540352   1.7   2.4      4.1  
65  ...  54  46  33  36  50  -73.830566  42.542080   1.0   1.2      2.2  

[5 rows x 22 columns]
(5764, 22)


In [62]:
df_solar_ml = pd.DataFrame()
df_solar_ml = df_solar[[
    'ANN', 'Land Value', 'Longitude', 'Latitude', 'Current'
    ]].query('ANN.notnull()') # SizeRank?
 
print(df_solar_ml.head())

   ANN     Land Value   Longitude   Latitude  Current
2   63  474416.339219 -116.327415  43.438301     94.6
3   63  474416.339219 -116.289497  43.468910     46.0
63  50  314400.663535  -73.865364  42.585896      4.3
64  50  314400.663535  -73.826706  42.540352      4.1
65  50  314400.663535  -73.830566  42.542080      2.2


In [63]:
print("df_solar            ...", df_solar.shape)
print("Current is not null ...", df_solar.query('Current.notnull()').shape)
print("Current is     null ...", df_solar.query('Current.isnull()' ).shape)

df_solar            ... (5764, 22)
Current is not null ... (5712, 22)
Current is     null ... (52, 22)


In [64]:
df_solar.query('Current.isnull()').shape

(52, 22)

## 5 &emsp; Splitting the DataFrame for Training and Testing

In [65]:
# df_solar_X_train \
#    = df_solar_ml.query('Current.notnull()').drop(['Current'], axis=1)

# df_solar_y_train \
#     = df_solar_ml.query('Current.notnull()')['Current']

X_train, X_test, y_train, y_test = train_test_split(
    df_solar_ml.query('Current.notnull()').drop(['Current'], axis=1),
    df_solar_ml.query('Current.notnull()')['Current'],
    random_state=42
    )

X_predict \
    = df_solar_ml.query('Current.isnull()' ).drop(['Current'], axis=1)

In [66]:
X_predict

,ANN,Land Value,Longitude,Latitude
90,60,2.277936e+05,NaN,NaN
93,48,1.915290e+05,NaN,NaN
95,43,3.924280e+05,NaN,NaN
174,55,2.065000e+05,NaN,NaN
199,62,3.171482e+05,NaN,NaN
228,63,1.959443e+05,NaN,NaN
469,55,2.729021e+05,NaN,NaN
495,59,3.262767e+05,NaN,NaN
544,58,1.461031e+05,NaN,NaN
548,67,2.115102e+05,NaN,NaN


## 6 &emsp; Predicting Current by Using Logistic Regression 

In [67]:
from sklearn.metrics import mean_squared_error

from sklearn.pipeline      import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model  import LinearRegression, Ridge
from sklearn.neighbors     import KNeighborsRegressor
from sklearn.tree          import DecisionTreeRegressor
from sklearn.svm           import SVR

from sklearn.ensemble import VotingRegressor

from sklearn.inspection import permutation_importance

## Try-it 20_1

In [68]:
reg_linear = LinearRegression()
reg_tree   = DecisionTreeRegressor(random_state=42)
reg_ridge  = Ridge()

# Create pipelines for each regressor
pipelines = {
    'LinearRegression()':      Pipeline([
        ('scaler',    StandardScaler()),
        ('regressor', reg_linear)
        ]),
    'KNeighborsRegressor()':   Pipeline([
        ('scaler',    StandardScaler()),
        ('regressor', KNeighborsRegressor())
        ]),
    'DecisionTreeRegressor()': Pipeline([
        ('regressor', reg_tree)
        ]),
    'Ridge()':        Pipeline([
        ('scaler',    StandardScaler()),
        ('regressor', reg_ridge)
        ]),
    'SVR()':                   Pipeline([
        ('scaler',    StandardScaler()),
        ('regressor', SVR())
        ])
}

# Define the Voting Regressor
voting_reg = VotingRegressor(estimators=[
    ('LinearRegression()',      pipelines['LinearRegression()']),
    ('KNeighborsRegressor()',   pipelines['KNeighborsRegressor()']),
    ('DecisionTreeRegressor()', pipelines['DecisionTreeRegressor()']),
    ('Ridge()',                 pipelines['Ridge()']),
    ('SVR()',                   pipelines['SVR()'])
])

# Function to evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    return mse

# Evaluate individual models without grid search
results_before_gs = {}
for name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    mse = evaluate_model(pipeline, X_test, y_test)
    results_before_gs[name] = mse

# Evaluate Voting Regressor without grid search
voting_reg.fit(X_train, y_train)
voting_mse_before_gs = evaluate_model(voting_reg, X_test, y_test)
results_before_gs['VotingRegressor()'] = voting_mse_before_gs

# Print results before grid search
print("Results Before Grid Search:")
for name, mse in results_before_gs.items():
    print(f'{name}: MSE = {mse}')

Results Before Grid Search:
LinearRegression(): MSE = 14908.24559595547
KNeighborsRegressor(): MSE = 14541.3172171123
DecisionTreeRegressor(): MSE = 19333.273422459893
Ridge(): MSE = 14908.339701162393
SVR(): MSE = 17264.974070634056
VotingRegressor(): MSE = 13566.934988825407


In [69]:
X_pred_voting_reg = voting_reg.predict(X_test)
pd.concat([
    X_test.reset_index(drop=True),
    y_test.reset_index(drop=True),
    pd.DataFrame(X_pred_voting_reg).reset_index(drop=True)
    ], axis=1)

,ANN,Land Value,Longitude,Latitude,Current,0
0,85,3.332105e+05,-110.973900,32.376358,9.6,59.834407
1,72,9.642500e+05,-118.128540,34.708324,3.2,17.048410
2,47,1.227581e+05,-81.751434,41.443275,7.7,8.996720
3,58,2.599504e+05,-72.650566,42.009109,4.9,20.646594
4,65,1.178752e+06,-158.063095,21.374256,25.0,24.897307
...,...,...,...,...,...,...
182,76,3.814220e+05,-120.343079,36.424538,46.3,133.363296
183,85,3.332105e+05,-110.891869,32.034618,265.0,58.535039
184,72,9.642500e+05,-118.191078,34.779060,6.9,78.554335
185,44,2.021899e+05,-75.958321,43.022732,12.4,1.678390


In [70]:
# Define parameter grids for Grid Search
param_grids = {
    'LinearRegression()':      {},
    'KNeighborsRegressor()':   {'regressor__n_neighbors': [3, 5, 7]},
    'DecisionTreeRegressor()': {'regressor__max_depth':   [3, 5, 7]},
    'Ridge()':                 {'regressor__alpha':       [0.1, 1.0, 10.0]},
    'SVR()':                   {
                                'regressor__C':           [0.1, 1.0, 10.0],
                                'regressor__gamma':       ['scale', 'auto']
        }
}

# Perform Grid Search and evaluate models
results_after_gs = {}
best_pipelines   = {}

for name, pipeline in pipelines.items():
    grid_search = GridSearchCV(
        pipeline, param_grids[name], cv=10, scoring='neg_mean_squared_error'
        )
    grid_search.fit(X_train, y_train)
    best_pipeline = grid_search.best_estimator_
    best_pipelines[name] = best_pipeline
    mse = evaluate_model(best_pipeline, X_test, y_test)
    results_after_gs[name] = mse

# Define the optimized Voting Regressor
optimized_voting_reg = VotingRegressor(estimators=[
    ('LinearRegression()',      best_pipelines['LinearRegression()']),
    ('KNeighborsRegressor()',   best_pipelines['KNeighborsRegressor()']),
    ('DecisionTreeRegressor()', best_pipelines['DecisionTreeRegressor()']),
    ('Ridge()',                 best_pipelines['Ridge()']),
    ('SVR()',                   best_pipelines['SVR()'])
])

# Fit the optimized Voting Regressor
optimized_voting_reg.fit(X_train, y_train)
voting_mse_after_gs = evaluate_model(optimized_voting_reg, X_test, y_test)
results_after_gs['VotingRegressor()'] = voting_mse_after_gs

# Print results after grid search
print("\nResults After Grid Search:")
for name, mse in results_after_gs.items():
    print(f'{name}: MSE = {mse}')


Results After Grid Search:
LinearRegression(): MSE = 14908.24559595547
KNeighborsRegressor(): MSE = 15015.098692810458
DecisionTreeRegressor(): MSE = 14355.609352091227
Ridge(): MSE = 14909.929849860975
SVR(): MSE = 16725.370063272545
VotingRegressor(): MSE = 13700.591274336293


---
Coefficient analysis

In [71]:
print("Feature Importance from Decision Tree:")
reg_tree.fit(X_train, y_train)

importances_tree = reg_tree.feature_importances_

feature_importance_tree_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': importances_tree
    })

feature_importance_tree_df = feature_importance_tree_df\
    .sort_values(by='importance', ascending=False)
print(feature_importance_tree_df)

Feature Importance from Decision Tree:
      feature  importance
2   Longitude    0.501636
3    Latitude    0.285450
0         ANN    0.152020
1  Land Value    0.060894


In [72]:
print("Coefficients from Linear Regression:")
reg_linear.fit(X_train, y_train)
coefficients_linear = reg_linear.coef_
coefficients_linear_df = pd.DataFrame({
    'feature': X_train.columns, 'coefficient': coefficients_linear
    })
coefficients_linear_df = coefficients_linear_df\
    .sort_values(by='coefficient', ascending=False)
print(coefficients_linear_df)

Coefficients from Linear Regression:
      feature  coefficient
0         ANN     2.765114
2   Longitude     0.167502
1  Land Value    -0.000063
3    Latitude    -1.925622


In [73]:
print("Coefficients from Ridge Regression:")

reg_ridge.fit(X_train, y_train)
coefficients_ridge = reg_ridge.coef_
coefficients_ridge_df = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': coefficients_ridge
    })
coefficients_ridge_df = coefficients_ridge_df\
    .sort_values(by='coefficient', ascending=False)

print(coefficients_ridge_df)

Coefficients from Ridge Regression:
      feature  coefficient
0         ANN     2.765061
2   Longitude     0.167450
1  Land Value    -0.000063
3    Latitude    -1.925409


In [74]:
print("Permutation Importance from KNeighborsRegressor():")
result_knn = permutation_importance(
    best_pipelines['KNeighborsRegressor()'], 
    X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )

perm_importances_knn = pd.DataFrame({
    'feature':    X_test.columns,
    'importance': result_knn.importances_mean
    })

perm_importances_knn = perm_importances_knn\
    .sort_values(by='importance', ascending=False)

print(perm_importances_knn)

Permutation Importance from KNeighborsRegressor():
      feature  importance
3    Latitude    1.003524
0         ANN    0.852624
2   Longitude    0.516412
1  Land Value    0.240275


In [75]:
print("Permutation Importance from SVR():")

result_svr = permutation_importance(
    best_pipelines['SVR()'],
    X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1
    )

perm_importances_svr = pd.DataFrame({
    'feature':    X_test.columns,
    'importance': result_svr.importances_mean
    })

perm_importances_svr = perm_importances_svr\
    .sort_values(by='importance', ascending=False)

print(perm_importances_svr)

Permutation Importance from SVR():
      feature  importance
0         ANN    0.045318
3    Latitude    0.005719
1  Land Value    0.003567
2   Longitude   -0.004472
